In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

In [ ]:
Image_size=256
Batch_size=32
Channels=3
Epochs=50

In [ ]:
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Train",
    shuffle=True,
    seed=123,
    image_size=(Image_size,Image_size),
    batch_size=Batch_size
)
class_names=train_dataset.class_names
train_dataset=train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Found 900 files belonging to 3 classes.


In [ ]:
valid_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Valid",
    shuffle=True,
    seed=12,
    image_size=(Image_size,Image_size),
    batch_size=Batch_size
)
valid_dataset=valid_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Found 300 files belonging to 3 classes.


In [ ]:
resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(Image_size,Image_size),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])
data_aug=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
train_dataset = train_dataset.map(
    lambda x, y: (data_aug(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
n_classes=3
model=models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(Batch_size,Image_size,Image_size,Channels)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])
model.build(input_shape=(Batch_size,Image_size,Image_size,Channels))

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history=model.fit(
    train_dataset,
    epochs=Epochs,
    batch_size=Batch_size,
    verbose=1,
    validation_data=valid_dataset
)

Epoch 1/50
29/29 [==============================] - 324s 8s/step - loss: 1.0713 - accuracy: 0.3933 - val_loss: 0.9713 - val_accuracy: 0.3933
Epoch 2/50
29/29 [==============================] - 171s 6s/step - loss: 0.9079 - accuracy: 0.5478 - val_loss: 0.7863 - val_accuracy: 0.5933
Epoch 3/50
29/29 [==============================] - 162s 6s/step - loss: 0.7105 - accuracy: 0.6911 - val_loss: 0.5553 - val_accuracy: 0.7767
Epoch 4/50
29/29 [==============================] - 164s 6s/step - loss: 0.5822 - accuracy: 0.7322 - val_loss: 0.8517 - val_accuracy: 0.5633
Epoch 5/50
29/29 [==============================] - 172s 6s/step - loss: 0.5607 - accuracy: 0.7567 - val_loss: 0.6019 - val_accuracy: 0.7000
Epoch 6/50
29/29 [==============================] - 162s 6s/step - loss: 0.4422 - accuracy: 0.8111 - val_loss: 0.5840 - val_accuracy: 0.7267
Epoch 7/50
29/29 [==============================] - 164s 6s/step - loss: 0.3703 - accuracy: 0.8289 - val_loss: 0.3790 - val_accuracy: 0.8200
Epoch 8/50
29

In [ ]:
def predict(img):
  img=tf.keras.preprocessing.image.img_to_array(img)
  img_arr=tf.expand_dims(img,0)
  pred=model.predict(img_arr)
  pred_class=class_names[np.argmax(pred[0])]
  confidence=round(100*(np.max(pred[0])),2)
  return pred_class, confidence

In [ ]:
model_version=3
model.save(f"/content/drive/MyDrive/Train_model/{model_version}")